In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import math
import time
import os
import nbimporter
from Class_FEA import *
import util_func
from itertools import product
from subprocess import Popen
import sys
from IPython.display import display, clear_output, Math, Latex

In [2]:
# Declare Abaqus job name, file path to job folder, and user subroutine name 
job_name = r"HGO_Tear_Propagation"
job_directory = r"D:\Users\Will\Tear_Propagation_Project"
umat_name = r"UMAT_HGO_C_Damage_Alpha"

# Declare work and data directories
work_directory = r"C:\Users\swilli9\Tear_Propagation_Project"
data_directory = r"{path}\Data_and_Visualizations".format(path=work_directory)


In [3]:
# Initialize class for handling Abaqus controls and data storage
FE = Abaqus(job_name, job_directory, data_directory)
# Class Attributes:
#
# FE.job, FE.job_path, FE.lines_init, FE.lines_inp, FE.coord, FE.connect, FE.num_nodes, FE.num_dof, FE.num_el,
# FE.Kc_0, FE.Kc_n, FE.Fc_n, FE.tear, FE.snapshots, FE.node_data, FE.el_data, FE.BC_mask
#
# Class Functions:
#
# FE.get_stiffness(mu), FE.get_max_load(mu), FE.init_FE_data_storage(), FE.get_FE_data(mu), FE.get_tear_len(mu)


# Declare run control file path
run_control = r"{folder}\run_input_file.bat".format(folder=work_directory)

init_file = r"{path}_INIT.inp".format(path = FE.job_path)
inp_file = r"{path}.inp".format(path = FE.job_path)
sta_file = r"{path}.sta".format(path = FE.job_path)

# Declare HGO constant parameters
HGO_constants =['C10 = {C10}\n'.format( C10 = 0.003 ), 
                'D1 = {D1}\n'.format(   D1  = 0.05  ), 
                'k1 = {k1}\n'.format(   k1  = 1.0   ), 
                'k2 = {k2}\n'.format(   k2  = 1.0   ), 
                'kap = {kap}\n'.format( kap = 0.25  ), 
                'N = {N}\n'.format(     N   = 2     ), 
                'T1 = {T1}\n'.format(   T1  = 1.35  ), #1.35
                'T2 = {T2}\n'.format(   T2  = 1.36  )] #1.36

c_ind = [FE.lines_inp.index(l) for l in FE.lines_inp if l.startswith('C10')][0]

FE.lines_inp[c_ind:c_ind+8]  = HGO_constants
FE.lines_init[c_ind:c_ind+8] = HGO_constants

with open(inp_file, "w") as file:
    file.writelines(FE.lines_inp)
    file.close()   
with open(init_file, "w") as file:
    file.writelines(FE.lines_init)
    file.close()

beta = np.array([[16.75, 40.78],
                 [26.11, 18.85],
                 [69.21, 63.2]])

beta_ind = [FE.lines_inp.index(l) for l in FE.lines_inp if 
            (l.startswith('beta1') or l.startswith('beta2') or l.startswith('beta3'))]


In [4]:
# Initialize wall clock time tracking
wall_clock = np.array([])

# Initialize/reset storage files for FE data if needed
reset = 1
if reset:
    check = input("You are about to overwrite data storage files for FE results. Are you sure? [y]/[n]: ")
    if check == 'y':
        FE.init_data_storage()
        clear_output()
    else:
        sys.exit("Script terminated. Adjust 'reset' flag if appending to existing data.")

# Iterate over beta parameter combinations, mu
for i, mu in enumerate(product(beta[0], beta[1], beta[2])):
    start = time.time()
    # Update beta values in the initial and main input files
    for j in range(3):
        FE.lines_inp[beta_ind[j]] = "beta{b} = {beta}\n".format(b=j+1,beta = mu[j])
        FE.lines_init[beta_ind[j]] = "beta{b} = {beta}\n".format(b=j+1,beta = mu[j])
    
    with open(inp_file, "w") as file:
        file.writelines(FE.lines_inp)
        file.close()   
    with open(init_file, "w") as file:
        file.writelines(FE.lines_init)
        file.close()
    
    # Set run control to initial input file
    with open(run_control, "r") as file:
        lines = file.readlines()
        file.close()
    lines[2] = r"abaqus job={job}_INIT user={umat} gpus=1 output_precision=full ask_delete=OFF".format(job=job_name,umat=umat_name)
    with open(run_control, "w") as file:
        file.writelines(lines)
        file.close()
    
    # Run initial stiffness simulation
    lock_file = r"{path}_INIT.lck".format(path = FE.job_path)
    p = Popen("run_input_file.bat", cwd=work_directory)
    stdout, stderr = p.communicate()
    
    # Wait for initial stiffness simulation to end
    time.sleep(5)
    while os.path.exists(lock_file):
        time.sleep(1)
    
    # Set run control to main input file
    with open(run_control, "r") as file:
        lines = file.readlines()
        file.close()
    lines[2] = r"abaqus job={job} user={umat} gpus=1 output_precision=full ask_delete=OFF".format(job=job_name,umat=umat_name)
    with open(run_control, "w") as file:
        file.writelines(lines)
        file.close()
    
    #Run main simulation
    lock_file = r"{path}.lck".format(path = FE.job_path)
    p = Popen("run_input_file.bat", cwd=work_directory)
    stdout, stderr = p.communicate()

    # Wait until status file is created
    time.sleep(5)
    while (not os.path.exists(sta_file)):
        time.sleep(1)

    # While main simulation is running, periodically check status file and display current increment and step time
    while os.path.exists(lock_file):
        last_line = util_func.read_n_to_last_line(sta_file)
        if last_line != "THE ANALYSIS HAS COMPLETED SUCCESSFULLY":
            line = last_line.split()
            clear_output(wait=True)
            display(Latex("$\mu_{i} = [{b1}^o,{b2}^o,{b3}^o]$".format(i=i+1,b1=mu[0],b2=mu[1],b3=mu[2])))
            display(Latex("\tIncrement: {inc}".format(inc=line[1]))) 
            display(Latex("\tStep Time: {t}".format(t=line[7])))
        time.sleep(10)
    
    # When main simulation finishes, output indication that the analysis is complete
    last_inc = util_func.read_n_to_last_line(sta_file, n=4).split()
    clear_output(wait=True)
    display(Latex("$\mu_{i} = [{b1}^o,{b2}^o,{b3}^o]$".format(i=i+1,b1=mu[0],b2=mu[1],b3=mu[2])))
    display(Latex("\tIncrement: {inc}".format(inc=last_inc[1]))) 
    display(Latex("\tStep Time: {t}".format(t=last_inc[7])))
    last_line = util_func.read_n_to_last_line(sta_file).strip()
    display("{complete}".format(complete=last_line))
    
    # Gather data from completed run
    FE.get_stiffness(mu)
    FE.get_max_load(mu)
    FE.get_data(mu)
    FE.get_tear_len(mu)
    
    end = time.time()
    
    wall_clock = np.append(wall_clock, end-start)

clock_file = r"{path}_wall_clock_time.txt".format(path=FE.data_path)
with open(clock_file, "w+") as file:
    file.write('\n'.join(str(wc) for wc in wall_clock))
    file.close()


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

'THE ANALYSIS HAS COMPLETED SUCCESSFULLY'

In [5]:
# terminate = 0

# if terminate == 0:
#     # Update the job termination control in case a job needs to be aborted mid-run
#     terminate = r"{folder}\terminate_job.bat".format(folder=work_directory)
#     with open(terminate, "r") as file:
#         lines = file.readlines()
#         file.close()   
#     lines[2] = r"abaqus terminate job={job}_INIT".format(job=job_name)
#     with open(terminate, "w") as file:
#         file.writelines(lines)
#         file.close()
#     p = Popen("terminate_job.bat", cwd=work_directory)
#     stdout, stderr = p.communicate()
    
# elif terminate == 1:
#     # Update the job termination control in case a job needs to be aborted mid-run
#     terminate = r"{folder}\terminate_job.bat".format(folder=work_directory)
#     with open(terminate, "r") as file:
#         lines = file.readlines()
#         file.close()   
#     lines[2] = r"abaqus terminate job={job}".format(job=job_name)
#     with open(terminate, "w") as file:
#         file.writelines(lines)
#         file.close()
#     p = Popen("terminate_job.bat", cwd=work_directory)
#     stdout, stderr = p.communicate()
